## Familias  que no estan en el core 90%

In [ ]:
##############################################################
# Script     : not_core.ipynb
# Author     : Johanna Atenea Carreón Baltazar
# Date       : 11/06/2025
# Description:
#     Este script analiza pangenomas de las bacterias de CAMDA 2024 
#     y CAMDA20 25 para identificar
#     familias de genes que no están presentes en todos los genomas,
#     basándose en un umbral definido.
#
#     Para cada bacteria:
#         - Calcula la frecuencia de presencia por familia.
#         - Separa familias core y no core.
#         - Guarda listas de familias no core.
#         - Filtra la matriz de pangenoma para conservar solo
#           las familias no core junto con las columnas base.
#
# Entradas requeridas:
#     - Archivos TSV de pangenomas en las carpetas:
#         2024_pangenomas/
#         2025_pangenomas/
#
# Salidas generadas:
#     - Frecuencias por familia:
#         2024_frecuencias/frecuencia_familia_vs_genomas_<bacteria>.csv
#         2025_frecuencias/frecuencia_familia_vs_genomas_<bacteria>.csv
#     - Listas de familias no core:
#         2024_not_core/not_core_familias_<bacteria>.csv
#         2025_not_core/not_core_familias_<bacteria>.csv
#     - Archivos de pangenomas filtrados:
#         2024_pangenomas_filtrados/filtrado_2024_pangenome_<bacteria>.csv
#         2025_pangenomas_filtrados/filtrado_2025_pangenome_<bacteria>.csv
#
# Requiere:
#     - pandas
#     - os
##############################################################


In [9]:
import pandas as pd
import os

In [10]:
def analizar_core_familias(nombre_bacteria: str, porcentaje_umbral: float, anio: int):
    """
    Analiza las familias de genes en un pangenoma y determina cuáles son core (comunes)
    y cuáles no, basándose en un umbral porcentual de presencia en genomas.

    La función realiza los siguientes pasos:
    - Carga una matriz de abundancia de familias de genes por genoma.
    - Calcula la frecuencia con la que cada familia está presente (valor > 0).
    - Aplica un umbral porcentual para definir qué familias son core.
    - Guarda:
        - Una tabla con la frecuencia de cada familia.
        - Una lista con las familias que no son core.
    - Imprime un resumen con el número de familias core y no core.

    Parameters
    ----------
    nombre_bacteria : str
        Nombre de la bacteria, usado para identificar archivos específicos.
    porcentaje_umbral : float
        Umbral mínimo de presencia (entre 0 y 1) requerido para considerar una familia como core.
        Por ejemplo, 0.95 significa "presente en al menos el 95% de los genomas".
    anio : int
        Año de referencia, usado para construir las rutas a los archivos de entrada y salida.

    Outputs
    -------
    Guarda archivos CSV en las siguientes rutas:
    - '{anio}_frecuencias/frecuencia_familia_vs_genomas_{nombre_bacteria}.csv'
    - '{anio}_not_core/not_core_familias_{nombre_bacteria}.csv'

    También imprime en consola un resumen del número de familias core y no core.
    """
    archivo = f"{anio}_pangenomas/{anio}_pangenome_{nombre_bacteria}.tsv"
    matriz = pd.read_csv(archivo, sep="\t")

    # Seleccionar solo las columnas de familias (desde la columna 8 en adelante)
    familias_df = matriz.iloc[:, 7:]

    # Contar el número de genomas únicos (columna 3, índice 2)
    total_genomas = matriz.iloc[:, 2].nunique()

    # Calcular en cuántos genomas está presente cada familia (abundancia > 0)
    frecuencia_por_familia = (familias_df > 0).sum(axis=0)

    frecuencia_df = pd.DataFrame({
        "familia": familias_df.columns,
        "frecuencia_genomas": frecuencia_por_familia.values
    })

    # Crear directorios si no existen
    os.makedirs(f"{anio}_frecuencias", exist_ok=True)
    os.makedirs(f"{anio}_not_core", exist_ok=True)

    frecuencia_df.to_csv(f"{anio}_frecuencias/frecuencia_familia_vs_genomas_{nombre_bacteria}.csv", index=False)

    umbral = int(porcentaje_umbral * total_genomas)

    # Separar familias en core,no core
    not_core_familias = frecuencia_df[(frecuencia_df["frecuencia_genomas"] < umbral) & (frecuencia_df["frecuencia_genomas"] != 0)]
    familias_sobrantes = (frecuencia_df["frecuencia_genomas"] == 0).sum()
    core_familias = frecuencia_df[frecuencia_df["frecuencia_genomas"] >= umbral]

    #Guardar familias que estan en el not core
    not_core_familias[['familia']].to_csv(f"{anio}_not_core/not_core_familias_{nombre_bacteria}.csv", index=False)

    
    
    # Imprimir resumen
    print(f"Número de familias presentes en al menos el {int(porcentaje_umbral * 100)}% de los genomas de {nombre_bacteria}: {len(core_familias)}")
    print(f"Número de familias presentes en menos de {int(porcentaje_umbral * 100)}% de los genomas de {nombre_bacteria}: {len(not_core_familias)}")
    print(f"Número de familias que no tienen presencia en ninguno de los genomas de {anio}: {familias_sobrantes}")


## Cortar pangenomas para conservar solo las familias que estan en el no core

In [11]:

def filtrar_familias_por_bacteria(anio: int, nombre_bacteria: str):
    """
    Filtra las columnas correspondientes a familias específicas (por ejemplo, not-core) 
    de un archivo de pangenoma, conservando también las primeras 7 columnas del archivo.

    Parámetros
    ----------
    anio : int
        Año utilizado como prefijo para las carpetas de entrada/salida.
    nombre_bacteria : str
        Nombre de la bacteria, usado para identificar el archivo CSV de familias a conservar.

    Estructura esperada
    -------------------
    - Carpeta de entrada: '{anio}_not_core', que contiene el archivo 'not_core_familias_{nombre_bacteria}.csv'
    - Archivo de pangenoma: '{anio}_pangenomas/{anio}_pangenome_{nombre_bacteria}.csv'
    - Carpeta de salida: '{anio}_pangenomas_filtrados'

    Guardará un archivo filtrado en la carpeta de salida, con nombre:
    - 'filtrado_{anio}_pangenome_{nombre_bacteria}.csv'
    """

    archivo_familias = f"{anio}_not_core/not_core_familias_{nombre_bacteria}.csv"
    familias_df = pd.read_csv(archivo_familias, header=None, names=["familia_descripcion"])
    familias_ids = familias_df["familia_descripcion"].tolist()

    carpeta_salida = f"{anio}_pangenomas_filtrados"
    
    archivo_pangenoma = f"{anio}_pangenomas/{anio}_pangenome_{nombre_bacteria}.tsv"
    df = pd.read_csv(archivo_pangenoma,sep='\t')

    # Conservar las primeras 7 columnas
    columnas_base = list(df.columns[:7])
    # A partir de la columna 8 (índice 7), buscar columnas que estén en la lista de familias
    columnas_familia = [col for col in df.columns[7:] if col in familias_ids]
    columnas_filtradas = columnas_base + columnas_familia

    df_filtrado = df[columnas_filtradas]
    salida = os.path.join(carpeta_salida, f"filtrado_{anio}_pangenome_{nombre_bacteria}.csv")
    df_filtrado.to_csv(salida, index=False)
    print(f"Procesado {nombre_bacteria}: {len(columnas_familia)} familias no conservadas.")
    print()

In [6]:
analizar_core_familias("acinetobacter", 0.90,2024)  # 90% de los genomas

Número de familias presentes en al menos el 90% de los genomas de acinetobacter: 2945
Número de familias presentes en menos de 90% de los genomas de acinetobacter: 25556
Número de familias que no tienen presencia en ninguno de los genomas de 2024: 41282


In [ ]:
acinetobacter = pd.read_csv('2024_pangenomas/2024_pangenome_acinetobacter_cut.tsv',sep='\t')

In [75]:
familia = acinetobacter['59280__Acetoin dehydrogenase E1 component alpha-subunit (EC 2.3.1.190)']
(familia != 0).sum()

809

In [ ]:
campylobacter =pd.read_csv('2024_pangenomas/2024_pangenome_campylobacter_cut.tsv',sep='\t')

In [ ]:
familia = acinetobacter['150_hypothetical protein']
(familia != 0).sum()

## CAMDA2024

In [6]:
bacterias2024 = ["acinetobacter", "campylobacter", "enterobacter", "escherichia", "klebsiella", "neisseria", "pseudomonas", "salmonella"]
for nombre in bacterias2024:
    analizar_core_familias(nombre, 0.9,2024)
    filtrar_familias_por_bacteria(2024,nombre)

Número de familias presentes en al menos el 90% de los genomas de acinetobacter: 2945
Número de familias presentes en menos de 90% de los genomas de acinetobacter: 25556
Número de familias que no tienen presencia en ninguno de los genomas de 2024: 41282
Procesado acinetobacter: 25556 familias no conservadas.



/tmp/ipykernel_2420943/1570115212.py:38: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  matriz = pd.read_csv(archivo, sep="\t")


Número de familias presentes en al menos el 90% de los genomas de campylobacter: 1433
Número de familias presentes en menos de 90% de los genomas de campylobacter: 6048
Número de familias que no tienen presencia en ninguno de los genomas de 2024: 5953


/tmp/ipykernel_2420943/1085778427.py:30: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo_pangenoma,sep='\t')


Procesado campylobacter: 6048 familias no conservadas.

Número de familias presentes en al menos el 90% de los genomas de enterobacter: 3517
Número de familias presentes en menos de 90% de los genomas de enterobacter: 21642
Número de familias que no tienen presencia en ninguno de los genomas de 2024: 0
Procesado enterobacter: 21642 familias no conservadas.



...

Procesado pseudomonas: 90416 familias no conservadas.



/tmp/ipykernel_2420943/1570115212.py:38: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  matriz = pd.read_csv(archivo, sep="\t")


Número de familias presentes en al menos el 90% de los genomas de salmonella: 3808
Número de familias presentes en menos de 90% de los genomas de salmonella: 37739
Número de familias que no tienen presencia en ninguno de los genomas de 2024: 8255


/tmp/ipykernel_2420943/1085778427.py:30: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo_pangenoma,sep='\t')


Procesado salmonella: 37739 familias no conservadas.



## CAMDA2025

In [12]:
bacterias2025 = ["acinetobacter","campylobacter","escherichia","klebsiella","neisseria","pseudomonas","salmonella","staphylococcus","streptococcus"]
for nombre in bacterias2025:
    analizar_core_familias(nombre, 0.9,2025)
    filtrar_familias_por_bacteria(2025,nombre)

/tmp/ipykernel_2420943/1570115212.py:38: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  matriz = pd.read_csv(archivo, sep="\t")


Número de familias presentes en al menos el 90% de los genomas de acinetobacter: 2926
Número de familias presentes en menos de 90% de los genomas de acinetobacter: 63171
Número de familias que no tienen presencia en ninguno de los genomas de 2025: 3686


/tmp/ipykernel_2420943/1085778427.py:30: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo_pangenoma,sep='\t')


Procesado acinetobacter: 63171 familias no conservadas.



/tmp/ipykernel_2420943/1570115212.py:38: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  matriz = pd.read_csv(archivo, sep="\t")


...

Procesado staphylococcus: 14899 familias no conservadas.



/tmp/ipykernel_2420943/1570115212.py:38: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  matriz = pd.read_csv(archivo, sep="\t")


Número de familias presentes en al menos el 90% de los genomas de streptococcus: 1632
Número de familias presentes en menos de 90% de los genomas de streptococcus: 6891
Número de familias que no tienen presencia en ninguno de los genomas de 2025: 498


/tmp/ipykernel_2420943/1085778427.py:30: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo_pangenoma,sep='\t')


Procesado streptococcus: 6891 familias no conservadas.



### Prueba con datos simulados

In [13]:
%autosave 30

Autosaving every 30 seconds
